In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ベースURL
base_url = "https://www.jalan.net/ikisaki/map/tokyo/"
response = requests.get(base_url)

# メタタグからエンコーディングを取得して設定
soup = BeautifulSoup(response.content, "html.parser")
meta_tag = soup.find("meta", {"charset": True})

if meta_tag and meta_tag.get("charset"):
    response.encoding = meta_tag.get("charset")
else:
    response.encoding = response.apparent_encoding

# 再度パース
soup = BeautifulSoup(response.text, "html.parser")

# 'map-right-inner' を持つ div を取得
map_right_inner_div = soup.find("div", class_="map-right-inner")
if map_right_inner_div:
    a_tags = map_right_inner_div.find_all("a", class_="sub")
    print(f"発見した a タグの数: {len(a_tags)}")
else:
    print("対象の div が見つかりませんでした。")
    a_tags = []

# URLを収集
urls = [urljoin(base_url, a['href']) for a in a_tags if 'href' in a.attrs]

# 特定の文字化けを修正する関数
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # 修正できない場合そのまま返す

# ページ内の全てのホテル情報を取得
def get_hotel_data_from_page(url):
    response = requests.get(url)

    # メタタグでエンコーディングを確認して設定
    soup = BeautifulSoup(response.content, "html.parser")
    meta_tag = soup.find("meta", {"charset": True})
    if meta_tag and meta_tag.get("charset"):
        response.encoding = meta_tag.get("charset")
    else:
        response.encoding = response.apparent_encoding

    # 再度HTMLをパース
    soup = BeautifulSoup(response.text, "html.parser")

    # ホテル情報を取得
    hotel_divs = soup.find_all("div", class_="p-yadoCassette__body p-searchResultItem__body")
    hotels = []

    for hotel_div in hotel_divs:
        # ホテル名
        h2_tag = hotel_div.find("h2", class_="p-searchResultItem__facilityName")
        raw_hotel_name = h2_tag.get_text(strip=True) if h2_tag else "ホテル名なし"
        hotel_name = fix_encoding(raw_hotel_name)

        # 詳細ページURL
        a_tag = hotel_div.find("a", class_="jlnpc-yadoCassette__link s16_00 fb")
        detail_url = a_tag['data-href'] if a_tag and 'data-href' in a_tag.attrs else "URLなし"

        # アクセス情報
        access_tag = hotel_div.find("dd", class_="p-searchResultItem__accessValue")
        access_info = access_tag.get_text(strip=True) if access_tag else "アクセス情報なし"

        # 価格情報
        price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestPriceValue")
        price_info = price_tag.get_text(strip=True) if price_tag else "価格情報なし"

        # 一人あたり価格
        unit_price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestUnitPrice")
        unit_price_info = unit_price_tag.get_text(strip=True) if unit_price_tag else "一人あたり価格情報なし"

        # ホテルの評価
        rating_tag = hotel_div.find("span", class_="p-searchResultItem__summaryaverage-num")
        rating_info = rating_tag.get_text(strip=True) if rating_tag else "評価なし"

        # レビュー数
        review_count_tag = hotel_div.find("span", class_="ji ji-bubble-outline")
        review_count_info = review_count_tag.get_text(strip=True) if review_count_tag else "レビュー数なし"

        # 情報を保存
        hotels.append({
            "ホテル名": hotel_name,
            "詳細ページURL": detail_url,
            "アクセス情報": access_info,
            "価格情報": price_info,
            "一人あたり価格": unit_price_info,
            "ホテルの評価": rating_info,
            "レビュー数": review_count_info
        })
    
    return hotels, soup

# 次ページを取得する関数
def get_next_page_url(soup, current_url):
    next_page_tag = soup.find("a", class_="jlnpc-paging__item-next")
    if next_page_tag and "href" in next_page_tag.attrs:
        return urljoin(current_url, next_page_tag["href"])
    return None

# 各URLについて処理
total_hotels_count = 0
for i, url in enumerate(urls, 1):
    print(f"URL {i}: {url}")
    current_url = url
    all_hotels = []

    while current_url:
        print(f"現在のページ: {current_url}")
        hotels, soup = get_hotel_data_from_page(current_url)
        all_hotels.extend(hotels)
        current_url = get_next_page_url(soup, current_url)

    # URLごとのホテル数を表示
    print(f"URL {i} で取得したホテルの数: {len(all_hotels)}")
    total_hotels_count += len(all_hotels)

    # 結果を出力（必要に応じて省略可）
    for hotel in all_hotels:
        print(hotel)
        print("-" * 50)

# 総数を表示
print(f"全てのURLから取得したホテルの総数: {total_hotels_count}")



発見した a タグの数: 13
URL 1: https://www.jalan.net/130000/LRG_137100/
現在のページ: https://www.jalan.net/130000/LRG_137100/
URL 1 で取得したホテルの数: 32
{'ホテル名': 'リッチモンドホテル東京芝', '詳細ページURL': "javascript:doYadDetailAd('358206','TGS01419110','115_1_1','2');", 'アクセス情報': 'JR・東京モノレール 浜松町駅 北口より徒歩約6分。浅草線・大江戸線 大門駅 A6出口より徒歩約2分。', '価格情報': '13,950円～', '一人あたり価格': '1名\xa06,975円～', 'ホテルの評価': '4.4', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': 'アパホテル〈品川 泉岳寺駅前〉', '詳細ページURL': "javascript:doYadDetailAd('354769','TGS01419110','115_1_2','2');", 'アクセス情報': '京急線・都営浅草線「泉岳寺駅」A2出口徒歩1分 ＪＲ「高輪ゲートウェイ駅」から徒歩4分', '価格情報': '9,800円～', '一人あたり価格': '1名\xa04,900円～', 'ホテルの評価': '4.2', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': 'ラビスタ東京ベイ（共立リゾート）', '詳細ページURL': 'URLなし', 'アクセス情報': '新交通 ゆりかもめ「市場前駅」より徒歩約１分', '価格情報': '26,800円～', '一人あたり価格': '1名\xa013,400円～', 'ホテルの評価': '4.6', 'レビュー数': ''}
--------------------------------------------------
{'ホテル名': 'ハートンホテル東品川（品川シーサイド）', '詳細ページURL': 'URLなし', 'アクセス情報

ConnectTimeout: HTTPSConnectionPool(host='www.jalan.net', port=443): Max retries exceeded with url: /130000/LRG_139500/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x11a5c9a50>, 'Connection to www.jalan.net timed out. (connect timeout=None)'))